In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)
import timeit

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes
# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Mon 03 Apr 2023 10:15:36 PM PDT
ndbs
Linux int-slurm-sarchive-p0005 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/big_data_tests


In [56]:
%load_ext autoreload
%autoreload 2

# Find data

Run of 18 seeds, reflecting a population of 18 million, from Rajan on 3/17/2023:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19
```

In [3]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/v1.2_parallelize_business_names'
    '/united_states_of_america/2023_03_16_17_21_09'
    '/final_results/2023_03_17_15_28_19'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 44K
drwxrwsr-x  5 rmudambi IHME-Simulationscience 2.0K Mar 17 16:06 ..
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 16:04 tax_dependents_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:59 tax_1040_observer
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Mar 17 15:53 .
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:51 tax_w2_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:50 social_security_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 wic_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_cps
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_acs
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:48 decennial_census_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience  18K Mar 17 15:29 logs


In [4]:
!du -s $output_dir

24417532	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [5]:
# Total of 24 GiB in all subdirectories
!du -sh $output_dir

24G	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [6]:
# 24 GiB = 25.77 GB
2**30/1e9 * 24

25.769803776000003

In [7]:
!ls -halt $output_dir/tax_w2_observer

total 12G
-rw-r--r--  1 rmudambi IHME-Simulationscience 669M Mar 17 15:58 tax_w2_observer_6817.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 665M Mar 17 15:58 tax_w2_observer_4344.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 670M Mar 17 15:58 tax_w2_observer_3541.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 679M Mar 17 15:57 tax_w2_observer_4507.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 674M Mar 17 15:57 tax_w2_observer_6545.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_781.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 677M Mar 17 15:57 tax_w2_observer_9672.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_8869.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 675M Mar 17 15:57 tax_w2_observer_6810.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_1362.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 682M Mar 17 15:

In [8]:
!ls -halt $output_dir/logs

total 296K
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:06 make_results_template_seed---seed_4344.o62940658_7
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6810.o62940658_13
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_3541.o62940658_6
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_4507.o62940658_8
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_8869.o62940658_16
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6545.o62940658_12
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6817.o62940658_14
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:04 make_results_template_seed---seed_781.o62940658_15
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:04 m

# See how `os.walk` works for iterating recursively through files in subdirectories

- https://docs.python.org/3/library/os.html#os.walk
- https://www.reddit.com/r/learnpython/comments/6yzgpm/osscandir_vs_oswalk_any_benefitdifference_of/
- https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right
- https://stackoverflow.com/questions/16953842/using-os-walk-to-recursively-traverse-directories-in-python

In [9]:
walker = os.walk(output_dir)
walker

<generator object walk at 0x7f2b61a0d200>

In [10]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19',
 ['decennial_census_observer',
  'tax_1040_observer',
  'wic_observer',
  'logs',
  'social_security_observer',
  'tax_dependents_observer',
  'household_survey_observer_cps',
  'tax_w2_observer',
  'household_survey_observer_acs'],
 [])

In [11]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/decennial_census_observer',
 [],
 ['decennial_census_observer_1362.csv.bz2',
  'decennial_census_observer_6545.csv.bz2',
  'decennial_census_observer_781.csv.bz2',
  'decennial_census_observer_5949.csv.bz2',
  'decennial_census_observer_4507.csv.bz2',
  'decennial_census_observer_3254.csv.bz2',
  'decennial_census_observer_6817.csv.bz2',
  'decennial_census_observer_4344.csv.bz2',
  'decennial_census_observer_6810.csv.bz2',
  'decennial_census_observer_2787.csv.bz2',
  'decennial_census_observer_9672.csv.bz2',
  'decennial_census_observer_5616.csv.bz2',
  'decennial_census_observer_5020.csv.bz2',
  'decennial_census_observer_9901.csv.bz2',
  'decennial_census_observer_3167.csv.bz2',
  'decennial_census_observer_2284.csv.bz2',
  'decennial_census_observer_3541.csv.bz2',
  'decennial_census_obs

In [12]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/tax_1040_observer',
 [],
 ['tax_1040_observer_6810.csv.bz2',
  'tax_1040_observer_9901.csv.bz2',
  'tax_1040_observer_781.csv.bz2',
  'tax_1040_observer_1362.csv.bz2',
  'tax_1040_observer_3254.csv.bz2',
  'tax_1040_observer_2284.csv.bz2',
  'tax_1040_observer_3541.csv.bz2',
  'tax_1040_observer_9672.csv.bz2',
  'tax_1040_observer_6545.csv.bz2',
  'tax_1040_observer_8869.csv.bz2',
  'tax_1040_observer_5020.csv.bz2',
  'tax_1040_observer_2787.csv.bz2',
  'tax_1040_observer_5616.csv.bz2',
  'tax_1040_observer_4344.csv.bz2',
  'tax_1040_observer_6817.csv.bz2',
  'tax_1040_observer_4507.csv.bz2',
  'tax_1040_observer_3167.csv.bz2',
  'tax_1040_observer_5949.csv.bz2'])

In [13]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/wic_observer',
 [],
 ['wic_observer_781.csv.bz2',
  'wic_observer_2284.csv.bz2',
  'wic_observer_5616.csv.bz2',
  'wic_observer_6817.csv.bz2',
  'wic_observer_1362.csv.bz2',
  'wic_observer_4344.csv.bz2',
  'wic_observer_3167.csv.bz2',
  'wic_observer_2787.csv.bz2',
  'wic_observer_6545.csv.bz2',
  'wic_observer_4507.csv.bz2',
  'wic_observer_5949.csv.bz2',
  'wic_observer_8869.csv.bz2',
  'wic_observer_9672.csv.bz2',
  'wic_observer_3541.csv.bz2',
  'wic_observer_6810.csv.bz2',
  'wic_observer_3254.csv.bz2',
  'wic_observer_9901.csv.bz2',
  'wic_observer_5020.csv.bz2'])

# Try loading a sample of each file using `os.walk`

In [46]:
%%time
dfs = {}
nrows = 1000
ext = '.csv.bz2'
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    print("="*50)
    print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            print("Loading shard", filename)
            %time shards.append(pd.read_csv(filepath, nrows=nrows))
    if shards:
        print("\nConcatenating data for ", observer_name)
        %time df = pd.concat(shards)
        dfs[observer_name] = df
    print("\n")
dfs = MappingViaAttributes(dfs)
dfs.keylist()

Loading data from 2023_03_17_15_28_19 



Loading data from decennial_census_observer 

Loading shard decennial_census_observer_1362.csv.bz2
CPU times: user 36.3 ms, sys: 4.83 ms, total: 41.1 ms
Wall time: 46.9 ms
Loading shard decennial_census_observer_6545.csv.bz2
CPU times: user 25.5 ms, sys: 3.84 ms, total: 29.3 ms
Wall time: 34.2 ms
Loading shard decennial_census_observer_781.csv.bz2
CPU times: user 24.4 ms, sys: 4.05 ms, total: 28.5 ms
Wall time: 33.9 ms
Loading shard decennial_census_observer_5949.csv.bz2
CPU times: user 28.5 ms, sys: 26 µs, total: 28.6 ms
Wall time: 33.5 ms
Loading shard decennial_census_observer_4507.csv.bz2
CPU times: user 29 ms, sys: 60 µs, total: 29.1 ms
Wall time: 34.5 ms
Loading shard decennial_census_observer_3254.csv.bz2
CPU times: user 29.2 ms, sys: 0 ns, total: 29.2 ms
Wall time: 33.6 ms
Loading shard decennial_census_observer_6817.csv.bz2
CPU times: user 27.1 ms, sys: 0 ns, total: 27.1 ms
Wall time: 31.8 ms
Loading shard decennial_census_observer_434

CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 25.6 ms
Loading shard social_security_observer_2284.csv.bz2
CPU times: user 19.9 ms, sys: 0 ns, total: 19.9 ms
Wall time: 24.5 ms

Concatenating data for  social_security_observer
CPU times: user 3.53 ms, sys: 0 ns, total: 3.53 ms
Wall time: 3.54 ms


Loading data from tax_dependents_observer 

Loading shard tax_dependents_observer_3254.csv.bz2
CPU times: user 25.5 ms, sys: 0 ns, total: 25.5 ms
Wall time: 30.5 ms
Loading shard tax_dependents_observer_5020.csv.bz2
CPU times: user 19.5 ms, sys: 3.72 ms, total: 23.2 ms
Wall time: 27.8 ms
Loading shard tax_dependents_observer_5616.csv.bz2
CPU times: user 20.2 ms, sys: 3.75 ms, total: 23.9 ms
Wall time: 28.1 ms
Loading shard tax_dependents_observer_2787.csv.bz2
CPU times: user 23.7 ms, sys: 249 µs, total: 24 ms
Wall time: 28.5 ms
Loading shard tax_dependents_observer_6810.csv.bz2
CPU times: user 23.4 ms, sys: 229 µs, total: 23.7 ms
Wall time: 28.1 ms
Loading shard tax_dependents_

CPU times: user 21.2 ms, sys: 74 µs, total: 21.3 ms
Wall time: 23.3 ms
Loading shard household_survey_observer_acs_9901.csv.bz2
CPU times: user 20.6 ms, sys: 34 µs, total: 20.7 ms
Wall time: 23 ms
Loading shard household_survey_observer_acs_6810.csv.bz2
CPU times: user 20.4 ms, sys: 18 µs, total: 20.4 ms
Wall time: 22.8 ms
Loading shard household_survey_observer_acs_781.csv.bz2
CPU times: user 20.4 ms, sys: 21 µs, total: 20.5 ms
Wall time: 22.7 ms
Loading shard household_survey_observer_acs_4344.csv.bz2
CPU times: user 20.2 ms, sys: 2 µs, total: 20.2 ms
Wall time: 22.7 ms
Loading shard household_survey_observer_acs_2787.csv.bz2
CPU times: user 20.4 ms, sys: 19 µs, total: 20.4 ms
Wall time: 22.8 ms
Loading shard household_survey_observer_acs_8869.csv.bz2
CPU times: user 21.3 ms, sys: 0 ns, total: 21.3 ms
Wall time: 23.6 ms
Loading shard household_survey_observer_acs_3541.csv.bz2
CPU times: user 17.4 ms, sys: 3.67 ms, total: 21 ms
Wall time: 23.4 ms
Loading shard household_survey_observe

['decennial_census_observer',
 'tax_1040_observer',
 'wic_observer',
 'social_security_observer',
 'tax_dependents_observer',
 'household_survey_observer_cps',
 'tax_w2_observer',
 'household_survey_observer_acs']

In [95]:
dfs.decennial_census_observer.simulant_id

0        1362_0
1        1362_1
2        1362_2
3        1362_3
4        1362_4
         ...   
995    8869_995
996    8869_996
997    8869_997
998    8869_998
999    8869_999
Name: simulant_id, Length: 18000, dtype: object

# Test my function

In [19]:
filepath

'/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/household_survey_observer_acs/household_survey_observer_acs_3167.csv.bz2'

In [20]:
%%time
df = datatypes.load_csv_data(filepath, convert_str_ids=True)
df

CPU times: user 461 ms, sys: 24.4 ms, total: 485 ms
Wall time: 622 ms


,middle_initial,simulant_id,household_id,first_name,street_number,last_name,housing_type,zipcode,survey_date,state,guardian_2,sex,unit_number,guardian_1,street_name,city,age,date_of_birth
0,D,3167000003457,3167000001404,Sheldon,1904,Meadows,Standard,30093.0,2020-04-29,GA,-1,Male,NaN,-1,gemstone dr,jonesboro,30.952562,1989-04-19
1,S,3167000003458,3167000001404,Hayley,1904,Meadows,Standard,30093.0,2020-04-29,GA,-1,Female,NaN,-1,gemstone dr,jonesboro,28.592003,1991-08-29
2,L,3167000003459,3167000001404,Annabelle,1904,Meadows,Standard,30093.0,2020-04-29,GA,3167000003458,Female,NaN,3167000003457,gemstone dr,jonesboro,1.574387,2018-09-04
3,A,3167000003460,3167000001404,Levi,1904,Meadows,Standard,30093.0,2020-04-29,GA,3167000003458,Male,NaN,3167000003457,gemstone dr,jonesboro,0.076660,2020-03-04
4,M,3167000008326,3167000003400,Johnny,5615,Crawford,Standard,43026.0,2020-04-29,OH,-1,Male,NaN,-1,north bell avenue,oregon,58.896404,1961-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19435,L,3167001100106,3167000277625,Maverick,510,Adsitt,Standard,68154.0,2030-05-15,NE,-1,Male,NaN,3167000689704,harmon av,omaha,4.381575,2025-12-27
19436,L,3167001143670,3167000327705,Declan,3248,Garcia Faustino,Standard,37308.0,2030-05-15,TN,-1,Male,NaN,3167000814381,ovlk wy,clarksville,1.934982,2028-06-08
19437,C,3167001152319,3167000485611,Brandon,6624,Roles,Standard,84061.0,2030-05-15,UT,-1,Male,apt 1,-1,e 115 st,spanish frk,40.575119,1989-10-17
19438,M,3167001168140,3167000152170,Ayla,20471,Rizzo,Standard,6371.0,2030-05-15,CT,3167000377594,Female,NaN,3167000377595,victoria ave,stamford,0.613333,2029-10-03


In [21]:
df.dtypes

middle_initial    category
simulant_id          int64
household_id         int64
first_name        category
street_number     category
last_name         category
housing_type      category
zipcode           category
survey_date         object
state             category
guardian_2           int64
sex               category
unit_number       category
guardian_1           int64
street_name       category
city              category
age                float32
date_of_birth     category
dtype: object

In [22]:
df.survey_date.nunique()

132

In [32]:
%%time
dtypes = {}
nrows = 100
ext = '.csv.bz2'
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
#     print("="*50)
#     print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
            break
    if shards:
#         print("\nConcatenating data for ", observer_name)
        df = pd.concat(shards)
        dtypes[observer_name] = df.dtypes
#     print("\n")
dtypes = MappingViaAttributes(dtypes)
dtypes.keylist()

CPU times: user 290 ms, sys: 8.09 ms, total: 298 ms
Wall time: 313 ms


['decennial_census_observer',
 'tax_1040_observer',
 'wic_observer',
 'social_security_observer',
 'tax_dependents_observer',
 'household_survey_observer_cps',
 'tax_w2_observer',
 'household_survey_observer_acs']

In [55]:
dtypes.household_survey_observer_cps

housing_type      category
state             category
street_name       category
age                float32
survey_date         object
zipcode           category
street_number     category
guardian_2           int64
date_of_birth     category
household_id         int64
simulant_id          int64
first_name        category
guardian_1           int64
sex               category
unit_number       category
city              category
middle_initial    category
last_name         category
dtype: object

In [45]:
[(k, col) for k, dt in dtypes.items() for col in dt.index if dt[col] == 'object']

[('tax_1040_observer', 'itin'),
 ('social_security_observer', 'event_type'),
 ('social_security_observer', 'event_date'),
 ('household_survey_observer_cps', 'survey_date'),
 ('household_survey_observer_acs', 'survey_date')]

In [48]:
dfs.tax_1040_observer.itin.unique()

array([nan, '941-65-9975', '906-87-4583', '992-62-4861', '935-80-0464',
       '996-79-9904', '915-76-5798', '942-61-7843', '985-61-5350',
       '976-56-8358', '967-64-7743', '980-63-8525', '915-81-6812',
       '943-90-4687', '976-57-3776', '968-50-5270', '921-77-8399',
       '973-55-6060', '994-57-3676', '993-94-9769', '913-65-3858',
       '929-70-9366', '971-84-9962', '997-55-3172', '982-50-9724',
       '960-59-1083', '901-58-8621', '976-77-0912', '956-95-5429',
       '907-62-1277', '986-71-4427', '921-80-7005', '920-96-8447',
       '926-83-2537', '955-51-2535', '922-99-3307', '997-63-1714',
       '928-72-7742', '947-86-5312', '955-81-8257', '964-90-3591',
       '988-75-1057', '935-94-5622', '921-80-8946', '931-92-1787',
       '959-97-2009', '940-75-2297', '903-80-5956', '983-65-3934',
       '993-84-0868', '918-54-6022', '962-86-3994', '922-54-2558',
       '937-83-1604', '915-54-2450', '911-52-9299', '984-84-5350',
       '982-98-0639', '961-75-9851', '917-71-2582', '972-

In [52]:
dfs.social_security_observer.event_type

0      creation
1      creation
2      creation
3      creation
4      creation
         ...   
995    creation
996    creation
997    creation
998    creation
999    creation
Name: event_type, Length: 18000, dtype: object

In [53]:
dfs.social_security_observer.event_date

0      1921-03-04
1      1921-03-04
2      1921-03-23
3      1921-04-01
4      1921-07-07
          ...    
995    1924-07-05
996    1924-07-06
997    1924-07-06
998    1924-07-06
999    1924-07-06
Name: event_date, Length: 18000, dtype: object

In [54]:
dfs.household_survey_observer_acs.survey_date

0      2020-04-29
1      2020-04-29
2      2020-04-29
3      2020-04-29
4      2020-04-29
          ...    
995    2020-10-14
996    2020-10-14
997    2020-10-14
998    2020-10-14
999    2020-10-14
Name: survey_date, Length: 18000, dtype: object

In [42]:
dtypes.tax_1040_observer['last_name'] == 'object'

False

In [43]:
'age' in dtypes.tax_1040_observer

True

# Concatenating dataframes with different categories results in non-categorical columns

In [59]:
%%time
dfs2 = {}
nrows = 100
ext = '.csv.bz2'
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
    if shards:
        df = pd.concat(shards)
        dfs2[observer_name] = df
dfs2 = MappingViaAttributes(dfs2)
dtypes2.keylist()

CPU times: user 5.23 s, sys: 99.9 ms, total: 5.33 s
Wall time: 5.41 s


['decennial_census_observer',
 'tax_1040_observer',
 'wic_observer',
 'social_security_observer',
 'tax_dependents_observer',
 'household_survey_observer_cps',
 'tax_w2_observer',
 'household_survey_observer_acs']

In [61]:
dfs2.decennial_census_observer.first_name

0     Herbert
1       Emery
2       Flora
3       Emily
4     Garrett
       ...   
95    Cynthia
96    Cameron
97       Ruth
98      Riley
99        Ian
Name: first_name, Length: 1800, dtype: object

# Try again, but explicitly set the categories before concatenating

In [62]:
fname_cat = dfs2.decennial_census_observer.first_name.astype('category')
fname_cat

0     Herbert
1       Emery
2       Flora
3       Emily
4     Garrett
       ...   
95    Cynthia
96    Cameron
97       Ruth
98      Riley
99        Ian
Name: first_name, Length: 1800, dtype: category
Categories (749, object): ['Alexander', 'Alyssa', 'Amanda', 'Amaya', ..., 'Yasmin', 'Yechiel', 'Youssef', 'Zachary']

In [63]:
fname_cat.cat.categories

Index(['Alexander', 'Alyssa', 'Amanda', 'Amaya', 'Amber', 'Amelia', 'Amy',
       'Andrea', 'Andrew', 'Angela',
       ...
       'Victor', 'Victoria', 'Vivian', 'Wanda', 'Wendy', 'William', 'Yasmin',
       'Yechiel', 'Youssef', 'Zachary'],
      dtype='object', length=749)

In [73]:
%%time
dfs3 = {}
nrows = 100
ext = '.csv.bz2'
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
    if shards:
        categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
        for col in categorical_cols:
            categories = shards[0][col].cat.categories
            for shard in shards[1:]:
                categories = categories.union(shard[col].cat.categories)
            for shard in shards:
                shard[col] = shard[col].cat.set_categories(categories)
        df = pd.concat(shards)
        dfs3[observer_name] = df
dfs3 = MappingViaAttributes(dfs3)
dfs3.keylist()

CPU times: user 5.4 s, sys: 67.6 ms, total: 5.47 s
Wall time: 5.55 s


['decennial_census_observer',
 'tax_1040_observer',
 'wic_observer',
 'social_security_observer',
 'tax_dependents_observer',
 'household_survey_observer_cps',
 'tax_w2_observer',
 'household_survey_observer_acs']

In [74]:
dfs3.decennial_census_observer.dtypes

first_name                    category
relation_to_household_head    category
guardian_1                       int64
zipcode                       category
street_name                   category
state                         category
guardian_2                       int64
race_ethnicity                category
unit_number                   category
simulant_id                      int64
housing_type                  category
last_name                     category
middle_initial                category
city                          category
date_of_birth                 category
street_number                 category
sex                           category
age                            float32
year                             int64
dtype: object

In [76]:
[(k, col) for k, df in dfs3.items() for col in df if df.dtypes[col] == 'object']

[]

In [77]:
[(k, col) for k, df in dfs3.items() for col in df if df.dtypes[col] == 'category']

[('decennial_census_observer', 'first_name'),
 ('decennial_census_observer', 'relation_to_household_head'),
 ('decennial_census_observer', 'zipcode'),
 ('decennial_census_observer', 'street_name'),
 ('decennial_census_observer', 'state'),
 ('decennial_census_observer', 'race_ethnicity'),
 ('decennial_census_observer', 'unit_number'),
 ('decennial_census_observer', 'housing_type'),
 ('decennial_census_observer', 'last_name'),
 ('decennial_census_observer', 'middle_initial'),
 ('decennial_census_observer', 'city'),
 ('decennial_census_observer', 'date_of_birth'),
 ('decennial_census_observer', 'street_number'),
 ('decennial_census_observer', 'sex'),
 ('tax_1040_observer', 'tax_year'),
 ('tax_1040_observer', 'mailing_address_po_box'),
 ('tax_1040_observer', 'mailing_address_city'),
 ('tax_1040_observer', 'date_of_birth'),
 ('tax_1040_observer', 'first_name'),
 ('tax_1040_observer', 'mailing_address_street_number'),
 ('tax_1040_observer', 'mailing_address_zipcode'),
 ('tax_1040_observer', 

# Do a test run saving the files

In [89]:
%%time
save_dir = '/share/scratch/users/ndbs/prl/temp'
observer_names = []
nrows = 1000
ext = '.csv.bz2'
seed_length=4
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    print("="*50)
    print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
#             seed = filename_root[:-seed_length]
            filepath = os.path.join(dirpath, filename)
            %time shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
    if shards:
        print("\nSetting categories and concatenating data for ", observer_name)
        categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
        for col in categorical_cols:
            categories = shards[0][col].cat.categories
            for shard in shards[1:]:
                categories = categories.union(shard[col].cat.categories)
            for shard in shards:
                shard[col] = shard[col].cat.set_categories(categories)
        df = pd.concat(shards, ignore_index=True)
        observer_names.append(observer_name)
        print("Saving data for ", observer_name)
        %time df.to_hdf( \
            f'{save_dir}/{observer_name}.hdf', key=observer_name, \
            format='table', complib='bzip2', complevel=9, \
        )
print(observer_names)
!ls -halt $save_dir

Loading data from 2023_03_17_15_28_19 

Loading data from decennial_census_observer 

CPU times: user 43.9 ms, sys: 11.7 ms, total: 55.6 ms
Wall time: 55.1 ms
CPU times: user 42 ms, sys: 3.86 ms, total: 45.8 ms
Wall time: 46.5 ms
CPU times: user 47.8 ms, sys: 0 ns, total: 47.8 ms
Wall time: 48.4 ms
CPU times: user 47.3 ms, sys: 0 ns, total: 47.3 ms
Wall time: 47.9 ms
CPU times: user 46.6 ms, sys: 0 ns, total: 46.6 ms
Wall time: 46.6 ms
CPU times: user 45 ms, sys: 0 ns, total: 45 ms
Wall time: 45.4 ms
CPU times: user 43.8 ms, sys: 0 ns, total: 43.8 ms
Wall time: 44.4 ms
CPU times: user 44.5 ms, sys: 30 µs, total: 44.5 ms
Wall time: 45.1 ms
CPU times: user 46.3 ms, sys: 158 µs, total: 46.5 ms
Wall time: 58.4 ms
CPU times: user 46.1 ms, sys: 0 ns, total: 46.1 ms
Wall time: 46.1 ms
CPU times: user 40.8 ms, sys: 3.96 ms, total: 44.8 ms
Wall time: 54 ms
CPU times: user 44.5 ms, sys: 33 µs, total: 44.6 ms
Wall time: 45.2 ms
CPU times: user 43.9 ms, sys: 0 ns, total: 43.9 ms
Wall time: 44.5 ms

CPU times: user 33.9 ms, sys: 0 ns, total: 33.9 ms
Wall time: 34.2 ms
CPU times: user 29.9 ms, sys: 7.51 ms, total: 37.4 ms
Wall time: 37.8 ms
CPU times: user 33.1 ms, sys: 0 ns, total: 33.1 ms
Wall time: 33.7 ms
CPU times: user 32.4 ms, sys: 0 ns, total: 32.4 ms
Wall time: 33.1 ms
CPU times: user 33.1 ms, sys: 0 ns, total: 33.1 ms
Wall time: 33.7 ms
CPU times: user 33.6 ms, sys: 53 µs, total: 33.6 ms
Wall time: 34.1 ms
CPU times: user 33.3 ms, sys: 83 µs, total: 33.4 ms
Wall time: 33.6 ms

Setting categories and concatenating data for  household_survey_observer_cps
Saving data for  household_survey_observer_cps
CPU times: user 384 ms, sys: 481 µs, total: 384 ms
Wall time: 408 ms
Loading data from tax_w2_observer 

CPU times: user 45 ms, sys: 65 µs, total: 45.1 ms
Wall time: 45.7 ms
CPU times: user 42.2 ms, sys: 141 µs, total: 42.3 ms
Wall time: 42.8 ms
CPU times: user 42.5 ms, sys: 0 ns, total: 42.5 ms
Wall time: 43.2 ms
CPU times: user 42.1 ms, sys: 0 ns, total: 42.1 ms
Wall time: 42

# Now see how long it takes to read them back in

In [93]:
%%time
dfs4 = {}
ext = '.hdf'
for dirpath, dirnames, filenames in os.walk(save_dir):
    print(dirpath, dirnames, filenames)
    for filename in filenames:
        if filename.endswith(ext):
            observer_name = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            print("\nLoading data from", observer_name)
            %time dfs4[observer_name] = pd.read_hdf(filepath)
            print(sizemb(dfs4[observer_name]), 'MB')

dfs4 = MappingViaAttributes(dfs4)
dfs4.keylist()

/share/scratch/users/ndbs/prl/temp [] ['tax_1040_observer.hdf', 'wic_observer.hdf', 'tax_dependents_observer.hdf', 'decennial_census_observer.hdf', 'tax_w2_observer.hdf', 'social_security_observer.hdf', 'household_survey_observer_acs.hdf', 'household_survey_observer_cps.hdf']

Loading data from tax_1040_observer
CPU times: user 222 ms, sys: 366 µs, total: 222 ms
Wall time: 249 ms
4.937435 MB

Loading data from wic_observer
CPU times: user 262 ms, sys: 3.2 ms, total: 266 ms
Wall time: 296 ms
6.931086 MB

Loading data from tax_dependents_observer
CPU times: user 216 ms, sys: 8.01 ms, total: 224 ms
Wall time: 253 ms
4.730205 MB

Loading data from decennial_census_observer
CPU times: user 216 ms, sys: 7.69 ms, total: 223 ms
Wall time: 250 ms
5.207217 MB

Loading data from tax_w2_observer
CPU times: user 349 ms, sys: 8.35 ms, total: 358 ms
Wall time: 402 ms
7.699659 MB

Loading data from social_security_observer
CPU times: user 94.5 ms, sys: 3.32 ms, total: 97.8 ms
Wall time: 111 ms
1.72057

['tax_1040_observer',
 'wic_observer',
 'tax_dependents_observer',
 'decennial_census_observer',
 'tax_w2_observer',
 'social_security_observer',
 'household_survey_observer_acs',
 'household_survey_observer_cps']

In [87]:
dfs.values()[1]

TypeError: 'ValuesView' object is not subscriptable